In [1]:
%%configure -f
{ "conf":{
          "spark.pyspark.python": "python3",
          "spark.pyspark.virtualenv.enabled": "true",
          "spark.pyspark.virtualenv.type":"native",
          "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv",
          "spark.sql.execution.arrow.enabled":"true"
         }
}

[I 2023-01-17 00:32:06,265.265 configure_magic] Magic cell payload received: {"conf": {"spark.pyspark.python": "python3", "spark.pyspark.virtualenv.enabled": "true", "spark.pyspark.virtualenv.type": "native", "spark.pyspark.virtualenv.bin.path": "/usr/bin/virtualenv", "spark.sql.execution.arrow.enabled": "true"}, "proxyUser": "assumed-role_fdp_blitvin-Isengard"}

[I 2023-01-17 00:32:06,265.265 configure_magic] Sending request to update kernel. Please wait while the kernel will be refreshed.


The kernel is successfully refreshed.

In [1]:
%pip install pyEX

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install awswrangler

     |████████████████████████████████| 1.1 MB 31.7 MB/s eta 0:00:01
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

aiobotocore 2.4.2 requires botocore<1.27.60,>=1.27.59, but you'll have botocore 1.29.50 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pyEX as p
import requests
import json
import pandas as pd
import numpy as np
#import vectorbt as vbt
import matplotlib.pyplot as plt 
import itertools 
import datetime 
import awswrangler as wr
import vectorbt as vbt
import pyspark.sql.functions as py_f
import pyspark.sql.types as py_t
isSandbox = False
if isSandbox:
    token ='Tpk_02dcd2036e7641b880dd4cbb01fa9c67'
    iex_ver = 'sandbox'
else:
    token ='pk_2e94555e43da4135a6032433c6b18fa5'
    iex_ver = 'stable'
pyEX_cl = p.Client(api_token=token)
timeframe='max'

/usr/local/lib/python3.7/site-packages/tzlocal/unix.py:177: UserWarning: Can not find any timezone configuration, defaulting to UTC.
  warnings.warn("Can not find any timezone configuration, defaulting to UTC.")


In [4]:
if False:
    fx_etf_symbols = ['FXA','FXY','FXE','FXB','UUP','FXC','FXF',]
    bitcoin_etf_symbols = ['BITO']
    commod_etf_symbols = ['USO','GLD','DBA','DBB','SLV',]
    fi_etf_symbols = ['IAGG','AGG','IHY','EMLC','HYG',]
    equity_etf_symbols = ['AMJ','AMLP','ARKF','ARKG','ARKK','ARKQ','ARKW','BOTZ','BUG','CGW','CIBR','COPX','CRBN','DRIV','EFV','EMLP','FBT','FDN','FIVG',
    'FIW','FTXG','GDX','GNR','GUNR','HACK','IAT','IBB','ICLN','IEO','IFRA','IGE','IGF','IGM','IGV','IHF','IHI','ITB','ITZ','IWD','IWN','IWS','IYT','JETS','KBWB',
                          'KOMP','KRE','KWEB','LCTU','LIT','MLPA',
    'MLPX','MOO','NFRA','OIH','PABU','PAVE','PBW','PEJ','PHO','PICK','PPA','QCLN','REET','REMX','ROBO','SCHH','SIL','SKYY','SMH','SOXX','SRVR','TAN','URA','URNM',
                          'VUG','XAR','XBI','XHB','XME','XOP','XSD','QQQ','EWZ','AAXJ','VTI','EWJ','EWA','EWC','MCHI','EWU','EWQ','EWG','FM','EIS']
    sectors = ['XLC','XLY','XLP','XLE','XLF','XLV','XLI','XLB','XLRE','XLK','XLU']
    bench_symbol=['SPY','QQQ','IVV']
    all_symbols =fx_etf_symbols+bitcoin_etf_symbols+commod_etf_symbols+fi_etf_symbols+equity_etf_symbols+bench_symbol+sectors
    #all_symbols
    columns_to_capture = ['close','open','symbol']
    data_folder = 's3://fsidatalake/eod_market_data/etf_trend_following/'
    parquet_file_out =f"{data_folder}/etf_market_data.parquet"

In [5]:
def get_full_file_name(data_folder,one_sym):
    return(f"{data_folder}{one_sym}.parquet")
def combine_new_and_stored_data(new_df,file_name,columns_to_capture):
    try:
        old_pd = pd.read_parquet(file_name)
        final_df=pd.concat([old_pd,new_df[columns_to_capture]]).drop_duplicates().sort_index()
    except Exception as l_exc:
        final_df=new_df.drop_duplicates().sort_index()
    return(final_df)
if False:
    md_df = None
    for one_sym in all_symbols:
        try:
            new_df = pyEX_cl.chartDF(symbol=one_sym, timeframe=timeframe)
            file_name = get_full_file_name(data_folder,one_sym)
            final_df = combine_new_and_stored_data(new_df,file_name,columns_to_capture)
            assert final_df['symbol'].values[0]==one_sym, 'STOP!!!'
            assert one_sym in file_name, 'STOP!!!'
            final_df[columns_to_capture].to_parquet(file_name,index=True)
            if md_df is None:
                md_df= spark.createDataFrame(final_df[columns_to_capture].reset_index())
            else:
                md_df=md_df.union(spark.createDataFrame(final_df[columns_to_capture].reset_index()))
        except Exception as l_exc:
            print(f"exc:{one_sym}, {l_exc}")
        #break
    md_df.write\
    .option("header",True)\
    .partitionBy('symbol')\
    .mode("overwrite")\
    .parquet(parquet_file_out)
    print(new_df.index.min(),new_df.index.max(),final_df.index.min(),final_df.index.max() )

NameError: name 'new_df' is not defined

In [335]:
ret_pd,max_date,hwm_l,local_sym,sym_pd=get_new_data_df(pd.DataFrame(['BITS'],columns=['symbol']))
sym_pd

panda UDF


,symbol,max_date,min_date,date_length
95,BITS,2023-01-13,2021-11-16,423 days


In [370]:
#@py_f.pandas_udf("date timestamp, close double, open double, symbol string", py_f.PandasUDFType.GROUPED_MAP)
#@py_f.pandas_udf("symbol string", py_f.PandasUDFType.GROUPED_MAP)
def get_new_data_df(symbol):
    local_sym=str(symbol.values[0][0])
    try:
        hwm_l=highwatermark_pd_bc.value
        columns_to_capture=columns_to_capture_bc.value
        iex_base_url=iex_base_url_bc.value
        sym_pd = hwm_l.query(f"symbol=='{local_sym}'")
        if len(sym_pd)>0:
            max_date = sym_pd['max_date'].iloc[0]
        else:
            max_date = datetime.datetime.now()-datetime.timedelta(days=365*50)
        last_days=(datetime.datetime.now()-max_date).days

        pyEX_cl = p.Client(api_token=iex_token_bc.value)
        final_url = iex_base_url.format(local_sym,iex_token_bc.value,int(last_days))
        #'https://cloud.iexapis.com/stable/stock/{}/chart/max?token={}&chartIEXOnly=true&chartLast={}'
        res = requests.get(final_url,headers=headers)
        ret_pd = pd.DataFrame(json.loads(res.text))#[columns_to_capture]
        ret_pd=ret_pd.query(f"date>'{max_date.strftime('%Y-%m-%d')}'")
        ret_pd['date']=pd.to_datetime(ret_pd['date'])
        print('panda UDF')
        #return(ret_pd[columns_to_capture].query(f"date>'{max_date}'"))
    except:
        ret_pd=pd.DataFrame([{'date': datetime.datetime.now()-datetime.timedelta(days=365*50),
                              'close': None, 
                              'open': None, 
                              'symbol': local_sym}])
    #return(ret_pd[columns_to_capture],max_date,hwm_l,local_sym,sym_pd)
    return(ret_pd[columns_to_capture])
    #return(pd.DataFrame([res.text],columns=['symbol']))
    
class MarketDataGateway():
    def __init__(self,isSandbox=False):
        self.fx_etf_symbols = ['FXA','FXY','FXE','FXB','UUP','FXC','FXF',]
        self.bitcoin_etf_symbols = ['BITO','BTF','XBTF','BITS','GBTC','BITW','BLOK','BLCN','LEGR','SPBC','BITQ','BKCH','DAPP','BTCFX']
        self.bitcoin_equity_proxy_symbols = ['COIN','MSTR']
        self.crypto_mining_symbols = ['RIOT','CAN','HUT','HIVE','MARA','BTCM','BTBT','BITF']
        self.commod_etf_symbols = ['USO','GLD','DBA','DBB','SLV',]
        self.fi_etf_symbols = ['IAGG','AGG','IHY','EMLC','HYG',]
        self.equity_etf_symbols = ['AMJ','AMLP','ARKF','ARKG','ARKK','ARKQ','ARKW','BOTZ','BUG','CGW','CIBR','COPX','CRBN','DRIV','EFV','EMLP','FBT','FDN','FIVG',
                'FIW','FTXG','GDX','GNR','GUNR','HACK','IAT','IBB','ICLN','IEO','IFRA','IGE','IGF','IGM','IGV','IHF','IHI','ITB','ITZ','IWD','IWN','IWS','IYT','JETS','KBWB',
                      'KOMP','KRE','KWEB','LCTU','LIT','MLPA',
                    'MLPX','MOO','NFRA','OIH','PABU','PAVE','PBW','PEJ','PHO','PICK','PPA','QCLN','REET','REMX','ROBO','SCHH','SIL','SKYY','SMH','SOXX','SRVR','TAN','URA','URNM',
                      'VUG','XAR','XBI','XHB','XME','XOP','XSD','QQQ','EWZ','AAXJ','VTI','EWJ','EWA','EWC','MCHI','EWU','EWQ','EWG','FM','EIS']
        self.sectors = ['XLC','XLY','XLP','XLE','XLF','XLV','XLI','XLB','XLRE','XLK','XLU']
        self.bench_symbol=['SPY','QQQ','IVV']
        self.all_symbols =self.fx_etf_symbols+\
                            self.bitcoin_etf_symbols+\
                            self.commod_etf_symbols+\
                            self.bitcoin_equity_proxy_symbols+\
                            self.crypto_mining_symbols+\
                            self.fi_etf_symbols+\
                            self.equity_etf_symbols+\
                            self.bench_symbol+\
                            self.sectors
        self.all_symbols_df = spark.createDataFrame(pd.DataFrame(self.all_symbols,columns=['symbol']))
        #all_symbols
        self.columns_to_capture = ['date','close','open','symbol']
        self.data_folder = 's3://fsidatalake/eod_market_data/etf_trend_following/'
        self.parquet_file_out =f"{self.data_folder}/etf_market_data.parquet"
        if isSandbox:
            self.iex_token ='Tpk_02dcd2036e7641b880dd4cbb01fa9c67'
            self.iex_ver = 'sandbox'
        else:
            self.iex_token ='pk_2e94555e43da4135a6032433c6b18fa5'
            self.iex_ver = 'stable'
        self.iex_base_url = 'https://cloud.iexapis.com/'+self.iex_ver+'/stock/{}/chart/max?token={}&chartIEXOnly=true&chartLast={}'
        
    def set_highwatermarks(self):
        try:
            self.old_df = spark.read.parquet(self.parquet_file_out)
        except:
            self.old_df = spark.createDataFrame(pd.DataFrame([{'symbol':'----','date':datetime.datetime.now()-datetime.timedelta(days=365*50)}]))
        self.highwatermark_pd = self.old_df.groupby('symbol').agg(py_f.max("date").alias("max_date"),
                             py_f.min("date").alias("min_date"),
                             (py_f.max("date")-py_f.min("date")).alias("date_length")
                            ).toPandas()
    def set_new_symbol_df(self):
        self.new_sym_df=self.all_symbols_df.groupby("symbol")\
                        .applyInPandas(get_new_data_df, schema="date timestamp, close double, open double, symbol string")\
                        .where('close is not null')
    def write_new_symbol_df(self):
        self.write_stats=self.new_sym_df.groupby('symbol').agg(py_f.count('symbol').alias('data_count'))
        self.new_sym_df.write\
            .option("header",True)\
            .partitionBy('symbol')\
            .mode("append")\
            .parquet(self.parquet_file_out)
    def rewrite_if_dups(self):
        self.old_df_after_write = spark.read.parquet(self.parquet_file_out)
        self.dup_df = self.old_df_after_write.distinct().groupby('date','symbol').count()
        dup_count=self.dup_df.where('count>1').count()
        if dup_count>0:        
            print(f"found:{dup_count} dups; sample:{self.dup_df.show()}")
            self.old_df_after_write.groupby('date','symbol').agg(py_f.mean('close').alias('close'),py_f.mean('open').alias('open'))\
            .write\
            .option("header",True)\
            .partitionBy('symbol')\
            .mode("overwrite")\
            .parquet(self.parquet_file_out.replace('.parquet','_temp.parquet'))
        else:
            print(f"no dups")
        

mdg=MarketDataGateway()
mdg.set_highwatermarks()
# set bc vars
highwatermark_pd_bc =sc.broadcast(mdg.highwatermark_pd)
columns_to_capture_bc =sc.broadcast(mdg.columns_to_capture)
iex_token_bc =sc.broadcast(mdg.iex_token)
iex_base_url_bc =sc.broadcast(mdg.iex_base_url)
# set bc vars
mdg.set_new_symbol_df()
mdg.write_new_symbol_df()
print(mdg.write_stats.toPandas().sort_values(['symbol']))
mdg.rewrite_if_dups()

Empty DataFrame
Columns: [symbol, data_count]
Index: []
no dups


In [357]:
mdg.old_df_after_write.distinct().groupby('date','symbol').count().where('count>1').count()#.orderBy(['count','symbol'],ascending=False).show()

5195

In [365]:
mdg.rewrite_if_dups()

+-------------------+------+-----+
|               date|symbol|count|
+-------------------+------+-----+
|2008-09-17 00:00:00|   EWZ|    1|
|2020-11-27 00:00:00|   PHO|    1|
|2021-11-01 00:00:00|   IGV|    1|
|2007-05-23 00:00:00|   VTI|    1|
|2022-02-15 00:00:00|  KOMP|    1|
|2021-05-04 00:00:00|  SRVR|    1|
|2018-12-31 00:00:00|   XHB|    1|
|2016-05-19 00:00:00|  HACK|    1|
|2015-04-08 00:00:00|  RIOT|    1|
|2020-11-20 00:00:00|  IFRA|    1|
|2022-06-07 00:00:00|  IAGG|    1|
|2018-06-12 00:00:00|  ARKW|    1|
|2018-10-19 00:00:00|  AAXJ|    1|
|2013-10-16 00:00:00|   SPY|    1|
|2011-07-06 00:00:00|   SPY|    1|
|2018-08-01 00:00:00|  MLPX|    1|
|2011-06-08 00:00:00|   FDN|    1|
|2020-03-06 00:00:00|   EWC|    1|
|2013-04-17 00:00:00|   SIL|    1|
|2016-07-14 00:00:00|   EWC|    1|
+-------------------+------+-----+
only showing top 20 rows

found:417044 dups; sample:None


Py4JJavaError: An error occurred while calling o9065.parquet.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.jobAbortedError(QueryExecutionErrors.scala:638)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:279)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:193)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:113)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:111)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.executeCollect(commands.scala:125)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$eagerlyExecuteCommands$1.$anonfun$applyOrElse$1(QueryExecution.scala:103)
	at org.apache.spark.sql.catalyst.QueryPlanningTracker$.withTracker(QueryPlanningTracker.scala:107)
	at org.

In [302]:
#lpd=mdg.highwatermark_pd
#sym='abc'
#sym_pd = lpd.query(f"symbol=='{sym}'")['max_date']
#if len(sym_pd)>0:
#    max_date = lpd.query(f"symbol=='{sym}'")['max_date'].iloc[0]
#else:
#    max_date = datetime.datetime.now()-datetime.timedelta(days=365*50)
#last_days=(datetime.datetime.now()-max_date).days
#res_pd = pyEX_cl.chartDF(symbol=sym, timeframe=timeframe,last=last_days).query(f"date>'{(max_date)}'")
res_pd.query(f"date>'{max_date.strftime('%Y-%m-%d')}'")

,close,high,low,open,priceDate,symbol,volume,id,key,subkey,...,uLow,uVolume,fOpen,fClose,fHigh,fLow,fVolume,label,change,changePercent
date,,,,,,,,,,,,,,,,,,,,,
2023-01-13,164.28,166.0100,164.1600,164.32,2023-01-13,ABC,1390667,HISTORICAL_PRICES,ABC,,...,164.1600,1390667,164.3200,164.2800,166.0100,164.1600,1390667,"Jan 13, 23",0.00,0.0000
2023-01-12,164.90,165.9650,164.1200,165.26,2023-01-12,ABC,1041570,HISTORICAL_PRICES,ABC,,...,164.1200,1041570,165.2600,164.9000,165.9650,164.1200,1041570,"Jan 12, 23",0.62,0.0038
2023-01-11,165.39,168.8400,164.7600,167.70,2023-01-11,ABC,1225794,HISTORICAL_PRICES,ABC,,...,164.7600,1225794,167.7000,165.3900,168.8400,164.7600,1225794,"Jan 11, 23",0.49,0.0030
2023-01-10,167.26,167.9500,164.9000,165.82,2023-01-10,ABC,1126569,HISTORICAL_PRICES,ABC,,...,164.9000,1126569,165.8200,167.2600,167.9500,164.9000,1126569,"Jan 10, 23",1.87,0.0113
2023-01-09,164.97,167.7200,164.9400,166.21,2023-01-09,ABC,1323620,HISTORICAL_PRICES,ABC,,...,164.9400,1323620,166.2100,164.9700,167.7200,164.9400,1323620,"Jan 9, 23",-2.29,-0.0137
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2007-01-08,46.45,46.6000,45.9000,46.21,2007-01-08,ABC,1844400,HISTORICAL_PRICES,ABC,,...,45.9000,1844400,37.7994,37.9958,38.1185,37.5459,1844400,"Jan 8, 07",0.15,0.0032
2007-01-05,45.73,46.4034,45.5600,46.11,2007-01-05,ABC,2005300,HISTORICAL_PRICES,ABC,,...,45.5600,2005300,37.7176,37.4068,37.9576,37.2677,2005300,"Jan 5, 07",-0.72,-0.0155
2007-01-04,46.29,46.4100,45.6501,46.23,2007-01-04,ABC,1390200,HISTORICAL_PRICES,ABC,,...,45.6501,1390200,37.8158,37.8649,37.9630,37.3414,1390200,"Jan 4, 07",0.56,0.0122


In [292]:
max_date.strftime("%Y-%m-%d")

datetime.datetime(1973, 1, 29, 4, 22, 11, 48817)

In [304]:

#base_url = 'https://cloud.iexapis.com/stable/stock/{}/chart/max?token={}&chartIEXOnly=true&chartLast={}'
#headers = {
#      'content-type': "application/json"
#  }

res = requests.get(base_url)
pd.DataFrame(json.loads(res.text))


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [109]:
(max_date-10).strftime('%Y-%m-%d')

TypeError: Addition/subtraction of integers and integer-arrays with Timestamp is no longer supported.  Instead of adding/subtracting `n`, use `n * obj.freq`

In [100]:
lpd

,symbol,max_date,min_date,date_length
0,IWN,2023-01-13,2006-12-29,5859 days
1,EIS,2023-01-13,2008-05-29,5342 days
2,EMLC,2023-01-13,2010-07-23,4557 days
3,XSD,2023-01-13,2007-03-01,5797 days
4,REMX,2023-01-13,2010-10-28,4460 days
...,...,...,...,...
119,URNM,2023-01-13,2019-12-04,1136 days
120,KOMP,2023-01-13,2018-10-23,1543 days
121,XLC,2023-01-13,2018-06-19,1669 days
122,FIVG,2023-01-13,2019-03-05,1410 days
